In [1]:
import copy
import math
import nbimporter
import numpy

import standard_house_area
import get_u_and_eta_from_u_a_and_eta_a as get_u_and_eta

Importing Jupyter notebook from standard_house_area.ipynb
Importing Jupyter notebook from nu.ipynb
Importing Jupyter notebook from get_u_and_eta_from_u_a_and_eta_a.ipynb
Importing Jupyter notebook from f.ipynb
Importing Jupyter notebook from H.ipynb


# 基準値計算用住戸の設定

## 1. 外皮性能に関する基準値の設定条件の概要

基準値計算のための条件設定の主な考え方は、以下のとおり。  

・外皮について、極力、UA値、ηA値の基準値を満たすように、基準値計算用の住宅のU値（仕様規定の部位別U値に比例）と窓のη値を決定する。    
・非現実的な設定は許容しない（例：窓の日射熱取得率が0未満）。

#### 基準値計算の方法  
①設計住戸の用途別面積を使用 (area_type: design_house_with_room_usage)  
　面積・長さ情報、用途別情報がない場合は推定。内壁情報は推定。   
②設計住戸の面積を使用 (area_type: design_house_without_room_usage)  
　面積・長さ情報がない場合は推定。内壁情報は推定。   
　用途別情報は、各部位の長さ・面積を室用途別の床面積に応じて各用途に割り当てる。   
③基準値計算用住戸を想定 (area_type: standard_house)  
　用途別床面積・外皮面積より基準値計算用住戸を決定。  
  
#### プラン  
①設計住戸  
②各用途が各方位に面する。  
③戸建は南側に居室。共同は南側に主居室、北側に他居室。  
  
#### 評価対象  
①外皮性能  
　プランは、周辺環境（隣棟・接道など）より決定される与条件とみなす  
②室配置と外皮性能  
③②同様  
  
#### 基準値計算用住戸の負荷の傾向の予想  
①設計住戸の負荷に比較的近い傾向を示すと考えられる。  
　プランによっては、基準値が大きめになる可能性がある。  
　例えば、北側リビングで開口部が北側に集中している場合、  
　断熱性能はUA値基準値を満たすように設定するが、  
　日射熱取得性能は非現実的な開口部（η>1）を許容しないとすると、  
　ηHが小さい条件で基準値計算をすることとなる可能性がある。  
　また、暖房時間が長い主居室が北側になるので、基準値が大きめになる。  
②暖房時間が長い主居室が南側のプランよりは負荷が大きく、  
　北側のプランよりは負荷が小さくなると考えられる。  
　開口部が北側に集中している場合、  
　断熱性能はUA値基準値を満たすように設定するが、  
　日射熱取得性能は非現実的な開口部（η>1）を許容しないとすると、  
　ηHが小さい条件で基準値計算をすることとなる可能性がある。  
③建物規模に応じて、概ね、負荷が決定される。  
　比較的妥当な設計を行った場合の負荷となる。  
　ηHの基準値を満たしやすい。  
  
#### 暖房の基準値を満たしやすい住戸の予想  
①暖房室の部位の断熱性能を高くした住戸  
　暖房室南側では、日射熱取得性能が高い住戸  
②室配置・部位性能・面積の工夫により、暖房室の断熱性能・日射熱取得性能を優先的に高くした住戸  
③②同様  
  
#### 入力情報LV1→LV2（面積・長さ情報の入力）  
①以下の場合、用途別情報を入力すると基準値をクリアしやすい。  
　－開口部面積率が標準住戸より小さい  
　－同部位の性能に差がある（例：方位別に適切に窓を選定している。断熱の弱い壁が部分的にある。）  
②①同様  
③面積・長さ情報を入力する場合、適切な設計では相対的に基準値をクリアしやすい。  
  
#### 入力情報LV2→LV3（用途別情報の入力）  
①同部位の性能が用途に依らない場合、用途別情報を入力する利点はない。  
②用途別情報を入力する場合、適切な設計では相対的に基準値をクリアしやすい。  
③②同様  

## 2. 外皮性能に関する基準値

エネルギーの使用の合理化に関する建築主等及び特定建築物の所有者の判断の基準の一部を改正する告示（平成28年4月1日施行）  
8地域の暖房期は、8地域の冷房期で代用

戸建住宅

| 　 | 1地域 | 2地域 | 3地域 | 4地域 | 5地域 | 6地域 | 7地域 | 8地域 |
|---|---|---|---|---|---|---|---|---|
| 平均熱貫流率[W/(㎡・K)] | 0.46 | 0.46 | 0.56 | 0.75 | 0.87 | 0.87 | 0.87 | 2.14 |
| 暖房期の平均日射熱取得率[-] | 2.5 | 2.3 | 2.7 | 3.7 | 4.5 | 4.3 | 4.6 | - |
| 冷房期の平均日射熱取得率[-] | 1.9 | 1.9 | 2.0 | 2.7 | 3.0 | 2.8 | 2.7 | 3.2 | 

共同住宅

| 　 | 1地域 | 2地域 | 3地域 | 4地域 | 5地域 | 6地域 | 7地域 | 8地域 |
|---|---|---|---|---|---|---|---|---|
| 平均熱貫流率[W/(㎡・K)] | 0.41 | 0.41 | 0.44 | 0.69 | 0.75 | 0.75 | 0.75 | 1.67 |
| 暖房期の平均日射熱取得率[-] | 1.5 | 1.3 | 1.5 | 1.8 | 2.1 | 2.0 | 2.1 | - |
| 冷房期の平均日射熱取得率[-] | 1.1 | 1.1 | 1.1 | 1.4 | 1.5 | 1.4 | 1.3 | 2.4 | 

#### Function

In [2]:
def get_std_UA(house_type, region):
    return {
        'detached'    : {1: 0.46, 2: 0.46, 3: 0.56, 4: 0.75, 5: 0.87, 6: 0.87, 7: 0.87, 8: 2.14 },
        'condominium' : {1: 0.41, 2: 0.41, 3: 0.44, 4: 0.69, 5: 0.75, 6: 0.75, 7: 0.75, 8: 1.67 }
    }[house_type][region]

In [3]:
def get_std_etaAH(house_type, region):
    return {
        'detached'    : {1: 2.5,  2: 2.3,  3: 2.7,  4: 3.7,  5: 4.5,  6: 4.3,  7: 4.6,  8: 3.2 },
        'condominium' : {1: 1.5,  2: 1.3,  3: 1.5,  4: 1.8,  5: 2.1,  6: 2.0,  7: 2.1,  8: 2.4 }
    }[house_type][region]

In [4]:
def get_std_etaAC(house_type, region):
    return {
        'detached'    : {1: 1.9,  2: 1.9,  3: 2.0,  4: 2.7,  5: 3.0,  6: 2.8,  7: 2.7,  8: 3.2 },
        'condominium' : {1: 1.1,  2: 1.1,  3: 1.1,  4: 1.4,  5: 1.5,  6: 1.4,  7: 1.3,  8: 2.4 }
    }[house_type][region]

## 3. 基準値計算用入力情報の統合

共通情報、外壁・開口部・内壁等の面積等を統合し、基準値計算用住宅のLv4の入力情報にコンバートする。

### 1) 共通

総外皮面積を追加する。

$$
\displaystyle
A_{total}=  \sum_{i}{A_i}
$$

$A_i$: 第$i$番目の外皮の部位の面積, m<sup>2</sup>  
$A_{total}$: 総外皮面積, m<sup>2</sup>  

#### Function

In [5]:
def get_total_area( d ):
    
    d_area = []
    for parts in ['general_parts','windows','doors','earthfloors'] :
        if (parts in d['envelope']) == True :
            d_area.extend(d['envelope'][parts])

    return sum(part['area'] for part in d_area)

In [6]:
def convert_common(d):
    
    d['common']['total_outer_skin_area'] = get_total_area(d)
    
    return d['common']

### 2) 外皮性能

外皮性能の関する基準値に関する情報を追加する。

#### Function

In [7]:
def convert_envelope_index(house_type, region):
    
    d = {}
    d['UA']    = get_std_UA(house_type, region)
    d['etaAH'] = get_std_etaAH(house_type, region)
    d['etaAC'] = get_std_etaAC(house_type, region)
    
    return d

### 3) 部位の設定

部位の性能値や層構成を設定する。

面積の取得

#### Function

In [8]:
def get_area(d_envelope, area_type):
    
    l_next_space        = ['outdoor','open_space','closed_space','open_underfloor','air_conditioned','closed_underfloor']
    l_direction         = ['top','n','ne','e','se','s','sw','w','nw','bottom','upward','horizontal','downward']
    l_space_type        = ['main_occupant_room','other_occupant_room','non_occupant_room','underfloor'] 
    l_general_part_type = ['roof','ceiling','wall','floor','boundary_wall','upward_boundary_flo','downward_boundary_floor']
    
    area = {}
    
    # 一般部位
    # 用途を考慮しない場合
    if area_type == 'design_house_without_room_usage' :
        area['general_parts'] = {
            v:{
                w: {
                    x: sum(y['area'] for y in d_envelope['general_parts'] \
                        if (y['general_part_type']==v and y['next_space']==w and y['direction']==x)) 
                    for x in l_direction
                } for w in l_next_space
            } for v in l_general_part_type
        }
    # 用途を考慮する場合
    else :
        area['general_parts'] = {
            v:{
                w: {
                    x: {
                        y: sum(z['area'] for z in d_envelope['general_parts'] \
                            if (z['general_part_type']==v and z['next_space']==w and z['direction']==x and z['space_type']==y))
                            for y in l_space_type
                    } for x in l_direction
                } for w in l_next_space
            } for v in l_general_part_type
        }
            
    # 窓
    # 用途を考慮しない場合   
    if area_type == 'design_house_without_room_usage' :
        area['windows'] = {
            w: {
                x: sum(y['area'] for y in d_envelope['windows'] \
                           if (y['next_space']==w and y['direction']==x)) for x in l_direction
            } for w in l_next_space
        }
    # 用途を考慮する場合
    else :
        area['windows'] = {
            w: {
                x: {
                    y: sum(z['area'] for z in d_envelope['windows'] \
                           if (z['next_space']==w and z['direction']==x and z['space_type']==y)) for y in l_space_type
                } for x in l_direction
            } for w in l_next_space
        }
    
    # ドア
    # 用途を考慮しない場合   
    if area_type == 'design_house_without_room_usage' :
        area['doors'] = {
            w: {
                x: sum(y['area'] for y in d_envelope['doors'] \
                           if (y['next_space']==w and y['direction']==x)) for x in l_direction
            } for w in l_next_space
        }
    # 用途を考慮する場合
    else :
        area['doors'] = {
            w: {
                x: {
                    y: sum(z['area'] for z in d_envelope['doors'] \
                           if (z['next_space']==w and z['direction']==x and z['space_type']==y)) for y in l_space_type
                } for x in l_direction
            } for w in l_next_space
        }

    # 土間床等の外周部
    # 用途を考慮しない場合   
    if area_type == 'design_house_without_room_usage' :
        area['earthfloor_perimeters'] = {
            w: {
                x: sum(y['length'] for y in d_envelope['earthfloor_perimeters'] \
                        if (y['next_space']==w and y['direction']==x)) \
                    for x in l_direction
            } for w in l_next_space
        }
    # 用途を考慮する場合
    else :
        area['earthfloor_perimeters'] = {
            w: {
                x: {
                    y: sum(z['length'] for z in d_envelope['earthfloor_perimeters'] \
                           if (z['next_space']==w and z['direction']==x and z['space_type']==y)) \
                        for y in l_space_type
                } for x in l_direction
            } for w in l_next_space
        }

    return area

一般部位のリスト作成

#### Function

In [9]:
def get_list_general_parts(area_general_parts, area_type) :
    
    l = []    
    
    # 用途を考慮しない場合 
    if area_type == 'design_house_without_room_usage' :
        for v in area_general_parts.keys() :
            for w in area_general_parts[v].keys() :
                for x in area_general_parts[v][w].keys() :
                    if area_general_parts[v][w][x] > 0 :
                        l.append({
                            'name'                  : v + '_' + (x if w =='outdoor' else w) ,
                            'general_part_type'     : v,
                            'next_space'            : w,
                            'external_surface_type' : ('outdoor' if w == 'outdoor' else None),
                            'direction'             : x,
                            'area'                  : area_general_parts[v][w][x],
                            'spec'                  : {
                                'structure': 'wood',
                                'u_value_input_method_wood': 'u_value_directly',
                                'is_sunshade_input': False
                            }
                        })
    
    # 用途を考慮する場合 
    else :
        for v in area_general_parts.keys() :
            for w in area_general_parts[v].keys() :
                for x in area_general_parts[v][w].keys() :
                    for y in area_general_parts[v][w][x].keys() :
                        if area_general_parts[v][w][x][y] > 0 :
                            l.append({
                                'name'                  : v + '_' + (x if w =='outdoor' else w) + '_' + y,
                                'general_part_type'     : v,
                                'next_space'            : w,
                                'external_surface_type' : ('outdoor' if w == 'outdoor' else None),
                                'direction'             : x,
                                'area'                  : area_general_parts[v][w][x][y],
                                'space_type'            : y,
                                'spec'                  : {
                                    'structure': 'wood',
                                    'u_value_input_method_wood': 'u_value_directly',
                                    'is_sunshade_input': False
                                }
                            })
    
    return l

窓のリスト作成

方位が北・北東・東・南東・南・南西・西・北西の窓については、日除け下端から窓上端までの垂直方向の距離：窓の開口高さ寸法：壁面からの日除けの張り出し寸法＝0:3:1とする。

#### Function

In [10]:
def get_list_windows(area_windows, area_type) :
    
    l = []    
    
    # 用途を考慮しない場合 
    if area_type == 'design_house_without_room_usage' :
        for x in area_windows.keys() :
            for y in area_windows[x].keys() :
                if area_windows[x][y] > 0 :
                    l.append({
                        'name'       : (y if x == 'outdoor' else x),
                        'next_space' :x,
                        'direction'  :y,
                        'area'       : area_windows[x][y],
                        'spec'       : {
                            'window_type'                : 'single',
                            'windows'                    : [{
                                'u_value_input_method'   : 'u_value_directly',
                                'eta_value_input_method' : 'eta_d_value_directly',
                                'glass_type'             : 'single'
                            }],
                            'attachment_type'            : 'none',
                            'is_windbreak_room_attached' : False,
                            'is_sunshade_input'          : True,
                                'sunshade': {
                                    'existance'    : True,
                                    'input_method' : 'simple',
                                    'd_h'          : 3 if y in ['n','ne','e','se','s','sw','w','nw'] else 0,
                                    'd_e'          : 0,
                                    'depth'        : 1 if y in ['n','ne','e','se','s','sw','w','nw'] else 0,
                                }                            
                        }
                    })

    # 用途を考慮する場合 
    else :
        for x in area_windows.keys() :
            for y in area_windows[x].keys() :
                for z in area_windows[x][y].keys() :
                    if area_windows[x][y][z] > 0 :
                        l.append({
                            'name'       : (y if x == 'outdoor' else x) + '_' + z,
                            'next_space' :x,
                            'direction'  :y,
                            'area'       : area_windows[x][y][z],
                            'space_type' : z,
                            'spec'       : {
                                'window_type'                : 'single',
                                'windows'                    : [{
                                    'u_value_input_method'   : 'u_value_directly',
                                    'eta_value_input_method' : 'eta_d_value_directly',
                                    'glass_type'             : 'single'
                                }],
                                'attachment_type'            : 'none',
                                'is_windbreak_room_attached' : False,
                                'is_sunshade_input'          : True,
                                    'sunshade': {
                                        'existance'    : True,
                                        'input_method' : 'simple',
                                        'd_h'          : 3 if y in ['n','ne','e','se','s','sw','w','nw'] else 0,
                                        'd_e'          : 0,
                                        'depth'        : 1 if y in ['n','ne','e','se','s','sw','w','nw'] else 0,
                                    }                            
                            }
                        })
                
    return l

ドアのリスト作成

#### Function

In [11]:
def get_list_doors(area_doors, area_type) :
    
    l = []    
    
    # 用途を考慮しない場合 
    if area_type == 'design_house_without_room_usage' :
        for x in area_doors.keys() :
            for y in area_doors[x].keys() :
                if area_doors[x][y] > 0 :
                    l.append({
                        'name'       : (y if x == 'outdoor' else x),
                        'next_space' :x,
                        'direction'  :y,
                        'area'       : area_doors[x][y],
                        'spec'       : {
                            'is_sunshade_input' : False
                        }
                    })

    # 用途を考慮する場合 
    else :
        for x in area_doors.keys() :
            for y in area_doors[x].keys() :
                for z in area_doors[x][y].keys() :
                    if area_doors[x][y][z] > 0 :
                        l.append({
                            'name'       : (y if x == 'outdoor' else x) + '_' + z,
                            'next_space' :x,
                            'direction'  :y,
                            'area'       : area_doors[x][y][z],
                            'space_type' : z,
                            'spec'       : {
                                'is_sunshade_input' : False
                            }
                        })
                
    return l

土間床等の外周部のリスト作成

#### Function

In [12]:
def get_list_earthfloor_perimeters(area_earthfloor_perimeters, area_type) :
    
    l = []   
    
    # 用途を考慮しない場合 
    if area_type == 'design_house_without_room_usage' :
        for x in area_earthfloor_perimeters.keys() :
            for y in area_earthfloor_perimeters[x].keys() :
                if area_earthfloor_perimeters[x][y] > 0 :
                    l.append({
                        'name'       : (y if x == 'outdoor' else x),
                        'length'     : area_earthfloor_perimeters[x][y],
                        'next_space' : x,
                        'direction'  : y,
                        'spec'       : {}
                    })

    # 用途を考慮する場合 
    else :
        for x in area_earthfloor_perimeters.keys() :
            for y in area_earthfloor_perimeters[x].keys() :
                for z in area_earthfloor_perimeters[x][y].keys() :
                    if area_earthfloor_perimeters[x][y][z] > 0 :
                        l.append({
                            'name'       : (y if x == 'outdoor' else x) + '_' + z,
                            'length'     : area_earthfloor_perimeters[x][y][z],
                            'space_type' : z,
                            'next_space' : x,
                            'direction'  : y,
                            'spec'       : {}
                        })
                
    return l

部位のリストの統合

#### Function

In [13]:
def get_list(area, area_type) :
    
    d = {}
    d['general_parts']         = get_list_general_parts(area['general_parts'], area_type)
    d['windows']               = get_list_windows(area['windows'], area_type)
    d['doors']                 = get_list_doors(area['doors'], area_type)
    d['earthfloor_perimeters'] = get_list_earthfloor_perimeters(area['earthfloor_perimeters'], area_type)
    
    return d['general_parts'], d['windows'], d['doors'], d['earthfloor_perimeters']

外壁・開口部・内壁の面積・性能値の設定

#### Function

In [14]:
def convert_parts(d, area_type):
    
    standard_d = {}
    
    # 部位面積の取得
    # 基準値計算用住戸を想定する場合
    if area_type == 'standard_house' :
        # 基準値計算用住戸の面積取得
        d_standard_area = {'envelope': {}}
        d_standard_area['envelope']['general_parts'], d_standard_area['envelope']['windows'], inner_walls = standard_house_area.get_area(d)
        d_standard_area['envelope']['doors'], d_standard_area['envelope']['earthfloor_perimeters'] =[], []
        
        standard_d['general_parts'], standard_d['windows'], standard_d['doors'], standard_d['earthfloor_perimeters'] \
            = get_list(get_area(d_standard_area['envelope'], area_type), area_type)
        
    # 設計住戸の面積を使用する場合
    elif area_type == 'design_house_with_room_usage' or area_type == 'design_house_without_room_usage':
        standard_d['general_parts'], standard_d['windows'], standard_d['doors'], standard_d['earthfloor_perimeters'] \
            = get_list(get_area(d['envelope'], area_type), area_type)

    # U値、η値の取得
    Eta, Eta_heating, Eta_cooling, part_U = get_u_and_eta.calc_adjustment_factor(d)
    
    # U値、η値の設定
    for part in standard_d['general_parts'] :
        part['spec']['u_value_wood'] = part_U[part['general_part_type']]
    for part in standard_d['windows'] :
        part['spec']['windows'][0]['u_vaue'] = part_U['window']
        part['spec']['windows'][0]['eta_d_value'] = Eta
    for part in standard_d['doors'] :
        part['spec']['u_vaue'] = part_U['door']
    for part in standard_d['earthfloor_perimeters'] :
        part['spec']['psi_value'] = part_U['earthfloor_perimeter']
        
    return standard_d['general_parts'], standard_d['windows'], standard_d['doors'], standard_d['earthfloor_perimeters']

土間床等のリスト作成

#### Function

In [15]:
def convert_earthfloor( d_earthfloors ) :
    
    d = copy.deepcopy(d_earthfloors)
                
    return d

### 4) 統合

#### Function

In [16]:
def convert(d, area_type):
    
    d['envelope']['index'] = convert_envelope_index(d['common']['house_type'], d['common']['region'])

    standard_d = {'envelope': {}}
    standard_d['common'] = convert_common(d)   
    standard_d['envelope']['general_parts'], standard_d['envelope']['windows'], standard_d['envelope']['doors'], \
        standard_d['envelope']['earthfloor_perimeters'] = convert_parts(d, area_type)
    if ('earthfloors' in d['envelope']) == True :
        standard_d['envelope']['earthfloors'] = convert_earthfloor(d['envelope']['earthfloors'])
    
    return standard_d

#### Example

In [21]:
d={
    'common': {
        'region': 6,
        'house_type': 'condominium', # 'detached','condominium'
        'main_occupant_room_floor_area': 98.0,
        'other_occupant_room_floor_area': 0.0,
        'total_floor_area': 98.0
    },
    'envelope': {
        'general_parts': [
            {'general_part_type': 'ceiling', 'area': 49, 'next_space': 'outdoor', 'direction': 'top',  'space_type': 'main_occupant_room'},
            {'general_part_type':    'wall', 'area': 40, 'next_space': 'outdoor', 'direction': 'n',  'space_type': 'main_occupant_room'},
            {'general_part_type':    'wall', 'area': 42, 'next_space': 'outdoor', 'direction': 'e',  'space_type': 'main_occupant_room'},
            {'general_part_type':    'wall', 'area': 22, 'next_space': 'outdoor', 'direction': 's',  'space_type': 'main_occupant_room'},
            {'general_part_type':    'wall', 'area': 42, 'next_space': 'outdoor', 'direction': 'w',  'space_type': 'main_occupant_room'},
            {'general_part_type':   'floor', 'area': 45, 'next_space': 'open_underfloor', 'direction': 'bottom', 'space_type': 'main_occupant_room'}
        ],
        'windows': [
            {'area': 20, 'next_space': 'outdoor', 'direction': 's',  'space_type': 'main_occupant_room'}
        ],
        'doors': [
            {'area': 2, 'next_space': 'outdoor', 'direction': 'n',  'space_type': 'main_occupant_room'}
        ],
        'earthfloor_perimeters': [
            {'length': 4, 'next_space': 'outdoor', 'direction': 'n',  'space_type': 'main_occupant_room'}
        ],
        'earthfloor_centers': [
            {'area': 4}
        ]
    }
}

convert(d, area_type = 'design_house_with_room_usage')

U値{'roof': 0.22499999999999998, 'ceiling': 0.22499999999999998, 'wall': 0.496875, 'floor': 0.4500000000000002, 'window': 4.359375, 'door': 4.359375, 'earthfloor_perimeter': 0.7125000000000004},開口部η0.44282273769772657,開口部ηH0.2723047075418278,開口部ηC0.44282273769772657
UA計算値0.75,ηAH計算値2.0,ηAC計算値1.4000000000000001


{'envelope': {'general_parts': [{'name': 'ceiling_top_main_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'top',
    'area': 49,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'is_sunshade_input': False,
     'u_value_wood': 0.22499999999999998}},
   {'name': 'wall_n_main_occupant_room',
    'general_part_type': 'wall',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'n',
    'area': 40,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'is_sunshade_input': False,
     'u_value_wood': 0.496875}},
   {'name': 'wall_e_main_occupant_room',
    'general_part_type': 'wall',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'e',
    'area': 42,
    'space_type': '